In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_KERAS'] = '1'

import pandas as pd
import numpy as np
import tensorflow as tf
import random
import pickle
import time
import keras
import gc

from tqdm import tqdm
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.metrics import *
from tensorflow.keras.layers import *
from tensorflow.keras.utils import *
from tensorflow.keras.losses import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.initializers import *
from tensorflow.keras.regularizers import *
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras import constraints
from keras_layer_normalization import LayerNormalization

from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import smart_cond
import tensorflow.keras.backend as K

# from keras_radam import RAdam
from keras_multi_head import MultiHeadAttention
# from keras_lookahead import Lookahead
# from keras_transformer import *

seed = 2020
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
random.seed(seed)
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

Using TensorFlow backend.


In [3]:
tf.test.is_gpu_available()

True

In [2]:
df = pd.read_pickle('/data/trans_df_4.pkl')
df.head()

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,0.000000,0.000000,0.133301,0.000000,0.416748,0.055542,0.199951,0.0,0.500000,...,9,4883,268,3,22746,0,3,92924,85,48299
1,0,0.000000,0.000000,0.333252,0.000000,0.250000,0.111084,0.000000,0.0,0.071411,...,0,3473,268,1,285940,0,3,31806,85,36596
2,0,0.000000,0.000000,0.000000,0.333252,0.375000,0.333252,0.066650,0.0,0.500000,...,6,1002,1,0,676979,12,3,27604,1,1
3,0,0.000000,0.529297,0.133301,0.111084,0.500000,0.222168,0.000000,0.0,0.000000,...,1,2289,1,0,0,0,3,68019,1,1
4,0,0.083313,0.000000,0.133301,0.000000,0.000000,0.000000,0.066650,0.0,0.000000,...,1,759,1,0,99687,0,2,84096,1,1


In [247]:
df.nunique()

label         2
I1           13
I2           17
I3           16
I4           10
I5           23
I6           19
I7           16
I8           13
I9           15
I10           4
I11           8
I12          12
I13          13
C1         1460
C2          561
C3       794520
C4       408541
C5          305
C6           23
C7        12311
C8          633
C9            3
C10       65275
C11        5463
C12      785111
C13        3187
C14          27
C15       13255
C16      670200
C17          10
C18        5128
C19        2133
C20           4
C21      754987
C22          18
C23          15
C24      120120
C25         100
C26       81747
dtype: int64

In [234]:
import time

time.time()

1645512064.6845841

In [237]:
a = """
> lrange 56_online_1645512064 0 10

1) "688"
2) "170"
3) "2715"
4) "2711"
5) "1957"
6) "49"
7) "55"
8) "630"
9) "3496"
10) "17"
"""

print(a)


> lrange 56_online_1645512064 0 10

1) "688"
2) "170"
3) "2715"
4) "2711"
5) "1957"
6) "49"
7) "55"
8) "630"
9) "3496"
10) "17"



In [231]:
pd.set_option('display.max_columns', None) 
df.head()

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,0.00,0.00,0.13,0.00,0.42,0.06,0.20,0.00,0.50,0.00,0.00,0.00,0.00,629,282,0,196621,43,11,10635,80,2,42964,3832,0,489,4,7188,358863,9,4883,268,3,22746,0,3,92924,85,48299
1,0,0.00,0.00,0.33,0.00,0.25,0.11,0.00,0.00,0.07,0.00,0.00,0.00,0.08,629,532,345334,103823,43,22,7057,24,2,11020,1660,301203,2697,18,11945,526435,0,3473,268,1,285940,0,3,31806,85,36596
2,0,0.00,0.00,0.00,0.33,0.38,0.33,0.07,0.00,0.50,0.00,0.14,0.09,0.42,228,29,8738,309108,43,11,9552,24,2,15031,2000,440882,2161,2,5675,141390,6,1002,1,0,676979,12,3,27604,1,1
3,0,0.00,0.53,0.13,0.11,0.50,0.22,0.00,0.00,0.00,0.00,0.00,0.00,0.08,629,102,0,73123,43,22,2262,24,2,61268,4911,0,674,2,1234,0,1,2289,1,0,0,0,3,68019,1,1
4,0,0.08,0.00,0.13,0.00,0.00,0.00,0.07,0.00,0.00,0.00,0.00,0.00,0.00,820,397,0,397510,43,3,8362,24,2,20980,4928,314635,3012,4,7407,4535,1,759,1,0,99687,0,2,84096,1,1


In [3]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('max_columns', 50)
df.head()

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,0.00,0.00,0.13,0.00,0.42,0.06,0.20,0.00,0.50,0.00,0.00,0.00,0.00,629,282,0,196621,43,11,10635,80,2,42964,3832,0,489,4,7188,358863,9,4883,268,3,22746,0,3,92924,85,48299
1,0,0.00,0.00,0.33,0.00,0.25,0.11,0.00,0.00,0.07,0.00,0.00,0.00,0.08,629,532,345334,103823,43,22,7057,24,2,11020,1660,301203,2697,18,11945,526435,0,3473,268,1,285940,0,3,31806,85,36596
2,0,0.00,0.00,0.00,0.33,0.38,0.33,0.07,0.00,0.50,0.00,0.14,0.09,0.42,228,29,8738,309108,43,11,9552,24,2,15031,2000,440882,2161,2,5675,141390,6,1002,1,0,676979,12,3,27604,1,1
3,0,0.00,0.53,0.13,0.11,0.50,0.22,0.00,0.00,0.00,0.00,0.00,0.00,0.08,629,102,0,73123,43,22,2262,24,2,61268,4911,0,674,2,1234,0,1,2289,1,0,0,0,3,68019,1,1
4,0,0.08,0.00,0.13,0.00,0.00,0.00,0.07,0.00,0.00,0.00,0.00,0.00,0.00,820,397,0,397510,43,3,8362,24,2,20980,4928,314635,3012,4,7407,4535,1,759,1,0,99687,0,2,84096,1,1


In [4]:
sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I' + str(i) for i in range(1, 14)]
use_features = sparse_features + dense_features
target = 'label'

In [157]:
fold_train_x, fold_valid_x, fold_train_y, fold_valid_y = train_test_split(
    df[use_features], df[target], stratify=df[target], test_size=0.2, random_state=seed)
fold_train_x.shape, fold_valid_x.shape, fold_train_y.shape, fold_valid_y.shape

((36672493, 39), (9168124, 39), (36672493,), (9168124,))

In [7]:
embed_list = df[sparse_features].nunique()
print(list(embed_list))

[1460, 561, 794520, 408541, 305, 23, 12311, 633, 3, 65275, 5463, 785111, 3187, 27, 13255, 670200, 10, 5128, 2133, 4, 754987, 18, 15, 120120, 100, 81747]


In [6]:
# input_embedding_dim_list = [1460, 583, 10131227, 2202608, 305, 24, 12517, 633, 3, 93145, 5683, 8351593, 3194, 27, 14992, 5461306, 10, 5652, 2173, 4, 7046547, 18, 15, 286181, 105, 142572]
input_embedding_dim_list = [1460, 561, 794520, 408541, 305, 23, 12311, 633, 3, 65275, 5463, 785111, 3187, 27, 13255, 670200, 10, 5128, 2133, 4, 754987, 18, 15, 120120, 100, 81747]
# input_embedding_dim_list = [1460, 570, 1545893, 668222, 305, 24, 12399, 633, 3, 74829, 5557, 1518204, 3194, 27, 13883, 1231447, 10, 5310, 2151, 4, 1437678, 18, 15, 157463, 104, 99817]
len(input_embedding_dim_list)

26

In [228]:
def reset_keras():
    K.clear_session()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
    sess = tf.Session(config=config)

    K.set_session(sess)
    gc.collect()
    
def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

class record_logs(Callback):
    def __init__(self, description, log_name):
        self.description = description
        self.log_name = log_name
        self.now_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        
    def on_train_begin(self, epoch, logs=None):
        with open(self.log_name, 'a+') as f:
            f.writelines(self.now_time + '\n')
            f.writelines(self.description + '\n')
    
    def on_epoch_begin(self, epoch, logs=None):
        self.start_time = time.time()
        
    def on_epoch_end(self, epoch, logs=None):
        self.end_time = time.time()
        self.last_time = self.end_time - self.start_time
        logs['time'] = self.last_time
        for i in logs:
            logs[i] = np.round(logs[i], 6)
        with open(self.log_name, 'a+') as f:
            f.writelines('epoch ' + str(epoch + 1) + ':' + str(logs) + '\n')

class gen_data(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
 
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))
 
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
 
        return np.array(batch_x), np.array(batch_y)

In [8]:
class Gate(Layer):
    def __init__(self, **kwargs):
        super(Gate, self).__init__(**kwargs)

    def build(self, input_shape):
        super(Gate, self).build(input_shape)  # Be sure to call this somewhere!

    def call(self, inputs, **kwargs):
        weight = Dense(1, activation='sigmoid', use_bias=False)(inputs)
        return inputs * weight

In [8]:
class FM(Layer):
    def __init__(self, **kwargs):
        super(FM, self).__init__(**kwargs)

    def build(self, input_shape):
        super(FM, self).build(input_shape)  # Be sure to call this somewhere!

    def call(self, inputs, **kwargs):
        concated_embeds_value = inputs
        square_of_sum = tf.square(tf.reduce_sum(
            concated_embeds_value, axis=1, keep_dims=True))
        sum_of_square = tf.reduce_sum(
            concated_embeds_value * concated_embeds_value, axis=1, keep_dims=True)
        cross_term = square_of_sum - sum_of_square
        cross_term = 0.5 * tf.reduce_sum(cross_term, axis=2, keep_dims=False)
        return cross_term

    def compute_output_shape(self, input_shape):
        return (None, 1)

In [12]:
class SENET(Layer):
    def __init__(self, reduction_ratio=3, seed=1024, **kwargs):
        self.reduction_ratio = reduction_ratio
        self.seed = seed
        super(SENET, self).__init__(**kwargs)

    def build(self, input_shape):
        self.field_size = input_shape[-2]
        self.embedding_size = input_shape[-1]
        self.reduction_size = max(1, self.field_size // self.reduction_ratio)

        self.W_0 = self.add_weight(shape=(
            self.field_size * self.embedding_size, self.field_size), initializer=glorot_normal(seed=self.seed), name="W_0")
        self.W_1 = self.add_weight(shape=(
            self.field_size, self.reduction_size), initializer=glorot_normal(seed=self.seed), name="W_1")
        self.W_2 = self.add_weight(shape=(
            self.reduction_size, self.field_size), initializer=glorot_normal(seed=self.seed), name="W_2")
        self.tensordot = tf.keras.layers.Lambda(
            lambda x: tf.tensordot(x[0], x[1], axes=(-1, 0)))
        super(SENET, self).build(input_shape)

    def call(self, inputs, training=None, **kwargs):
        # Z = tf.reduce_mean(inputs, axis=-1)
        Z = Flatten()(inputs)
        # Z = Conv1D(filters=self.embedding_size, kernel_size=1, strides=1, padding='valid')(inputs)
        # Z = tf.reduce_mean(Z, axis=-1, keepdims=False)
        Z = tf.nn.relu(self.tensordot([Z, self.W_0]))
        Z = tf.nn.sigmoid(self.tensordot([Z, self.W_1]))
        Z = tf.nn.sigmoid(self.tensordot([Z, self.W_2]))
        V = tf.multiply(inputs, tf.expand_dims(Z, axis=2))
        return V
        # return tf.split(V, self.field_size, axis=1)
        
        
# class DNNSENET(Layer):
#     def __init__(self, reduction_ratio=2, seed=1024, feature_size=256, **kwargs):
#         self.feature_size = feature_size
#         self.reduction_ratio = reduction_ratio
#         self.seed = seed
#         super(DNNSENET, self).__init__(**kwargs)

#     def build(self, input_shape):
#         self.embedding_size = input_shape[-1]
#         self.reduction_size = max(1, self.embedding_size // self.reduction_ratio)

#         self.W_0 = self.add_weight(shape=(
#             self.embedding_size, self.reduction_size), initializer=glorot_normal(seed=self.seed), name="W_0")
#         self.W_1 = self.add_weight(shape=(
#             self.reduction_size, self.embedding_size), initializer=glorot_normal(seed=self.seed), name="W_1")
#         self.tensordot = tf.keras.layers.Lambda(
#             lambda x: tf.tensordot(x[0], x[1], axes=(-1, 0)))
#         super(DNNSENET, self).build(input_shape)

#     def call(self, inputs, training=None, **kwargs):
#         Z = inputs
#         Z = tf.nn.relu(self.tensordot([Z, self.W_0]))
#         Z = tf.nn.sigmoid(self.tensordot([Z, self.W_1]))
#         return Z

In [169]:
class CrossNet_V1(Layer):
    def __init__(self, layer_num=2, l2_reg=0, seed=1024, **kwargs):
        self.layer_num = layer_num
        self.l2_reg = l2_reg
        self.seed = seed
        super(CrossNet_V1, self).__init__(**kwargs)

    def build(self, input_shape):
        dim = int(input_shape[-1])
        self.kernels = [self.add_weight(name='kernel' + str(i),
                                        shape=(dim, 1),
                                        initializer=glorot_normal(
                                            seed=self.seed),
                                        regularizer=l2(self.l2_reg),
                                        trainable=True) for i in range(self.layer_num)]
        self.bias = [self.add_weight(name='bias' + str(i),
                                     shape=(dim, 1),
                                     initializer=Zeros(),
                                     trainable=True) for i in range(self.layer_num)]
        super(CrossNet_V1, self).build(input_shape)

    def call(self, inputs, **kwargs):
        x_0 = tf.expand_dims(inputs, axis=2)
        x_l = x_0
        for i in range(self.layer_num):
            xl_w = tf.tensordot(x_l, self.kernels[i], axes=(1, 0))
            dot_ = tf.matmul(x_0, xl_w)
            x_l = dot_ + self.bias[i] + x_l
        x_l = tf.squeeze(x_l, axis=2)
        return x_l
    

class CrossNet_V2(Layer):
    def __init__(self, layer_num=3, l2_reg=0, seed=1024, **kwargs):
        self.layer_num = layer_num
        self.l2_reg = l2_reg
        self.seed = seed
        super(CrossNet_V2, self).__init__(**kwargs)

    def build(self, input_shape):
        dim = int(input_shape[-1])
        self.kernels = [self.add_weight(name='kernel' + str(i),
                                        shape=(dim, dim),
                                        initializer=glorot_normal(seed=self.seed),
                                        regularizer=l2(self.l2_reg),
                                        trainable=True) for i in range(self.layer_num)]
        self.bias = [self.add_weight(name='bias' + str(i),
                                     shape=(dim, 1),
                                     initializer=Zeros(),
                                     trainable=True) for i in range(self.layer_num)]
        super(CrossNet_V2, self).build(input_shape)

    def call(self, inputs, **kwargs):
        x_0 = tf.expand_dims(inputs, axis=2)
        x_l = x_0
        for i in range(self.layer_num):
            wl_xl = tf.matmul(self.kernels[i], x_l)
            x_m = wl_xl + self.bias[i]
            x_l = x_0 * x_m + x_l
        x_l = tf.squeeze(x_l, axis=2)
        return x_l

    def get_config(self, ):
        config = {'layer_num': self.layer_num,
                  'l2_reg': self.l2_reg, 'seed': self.seed}
        base_config = super(CrossNet_V2, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape

In [66]:
class CrossNetMix(Layer):
    def __init__(self, low_rank=32, num_experts=6, layer_num=2, seed=2020):
        self.low_rank = low_rank
        self.num_experts = num_experts
        self.layer_num = layer_num
        self.seed = seed
        super(CrossNetMix, self).__init__()

    def build(self, input_shape):
        # U: (in_features, low_rank)
        self.in_features = int(input_shape[1])
        self.U_list = [self.add_weight(name='kernel_U' + str(i),
                                       shape=(self.num_experts, self.in_features, self.low_rank),
                                       initializer=glorot_normal(seed=self.seed))
                                       for i in range(self.layer_num)]
        # V: (in_features, low_rank)
        self.V_list = [self.add_weight(name='kernel_V' + str(i),
                                       shape=(self.num_experts, self.in_features, self.low_rank),
                                       initializer=glorot_normal(seed=self.seed))
                                       for i in range(self.layer_num)]

        # C: (low_rank, low_rank)
        self.C_list = [self.add_weight(name='kernel_C' + str(i),
                                       shape=(self.num_experts, self.low_rank, self.low_rank),
                                       initializer=glorot_normal(seed=self.seed))
                                       for i in range(self.layer_num)]

        self.gating = [Dense(1, use_bias=False) for i in range(self.num_experts)]

        self.bias = [self.add_weight(name='bias' + str(i),
                                    shape=(self.in_features, 1),
                                    initializer=Zeros()) 
                                    for i in range(self.layer_num)]
        super(CrossNetMix, self).build(input_shape)

    def call(self, inputs, **kwargs):
        # (bs, in_features, 1)
        x_0 = tf.expand_dims(inputs, 2)  
        x_l = x_0
        for i in range(self.layer_num):
            output_of_experts = []
            gating_score_of_experts = []
            for expert_id in range(self.num_experts):
                # (bs, in_features) -> (bs, 1)
                gating_score_of_experts.append(self.gating[expert_id](tf.squeeze(x_l, 2)))

                # (bs, in_features, 1) -> (bs, low_rank, 1)
                v_x = tf.matmul(tf.transpose(self.V_list[i][expert_id]), x_l)

                v_x = tf.tanh(v_x)
                
                # (bs, low_rank, 1) -> (bs, low_rank, 1)
                v_x = tf.matmul(self.C_list[i][expert_id], v_x)
                v_x = tf.tanh(v_x)

                # (bs, low_rank, 1) -> (bs, in_features, 1)
                uv_x = tf.matmul(self.U_list[i][expert_id], v_x)  

                dot_ = uv_x + self.bias[i]
                
                # (bs, in_features, 1)
                dot_ = x_0 * dot_ 

                output_of_experts.append(tf.squeeze(dot_, 2))

            # (bs, in_features, num_experts)
            output_of_experts = tf.stack(output_of_experts, 2)  
            # (bs, num_experts, 1)
            gating_score_of_experts = tf.stack(gating_score_of_experts, 1)  
            gating_value = tf.nn.softmax(gating_score_of_experts, 1)
            
            moe_out = tf.matmul(output_of_experts, gating_value)
            x_l = moe_out + x_l  # (bs, in_features, 1)

        x_l = tf.squeeze(x_l, -1)  # (bs, in_features)
        return x_l

In [15]:
class CrossFM(Layer):
    def __init__(self, **kwargs):
        super(CrossFM, self).__init__(**kwargs)

    def build(self, input_shape):
        self.field_size = input_shape[-2]
        self.embedding_size = input_shape[-1]
        super(CrossFM, self).build(input_shape)  # Be sure to call this somewhere!
        
    def call(self, inputs, **kwargs):
        x = inputs
        result = []
        for i in range(self.field_size):
            cur = x[:, i]
            other = tf.concat([x[:, :i], x[:, i:]], axis=1)
            other_embed = Dense(self.embedding_size, activation='relu')(Flatten()(other))
            res = tf.reduce_sum(cur * other_embed, axis=1, keepdims=True)
            result.append(res)
        result = tf.concat(result, axis=1)
        return result
    
    
class WeightedFM(Layer):
    def __init__(self, **kwargs):
        super(WeightedFM, self).__init__(**kwargs)

    def build(self, input_shape):
        self.field_size = input_shape[-2]
        self.embedding_size = input_shape[-1]
        super(WeightedFM, self).build(input_shape)  # Be sure to call this somewhere!
        
    def call(self, inputs, **kwargs):
        x = inputs
        result = []
        for i in range(self.field_size):
            cur = x[:, i]
            other = tf.concat([x[:, :i], x[:, i:]], axis=1)
            other_embed = Dense(self.embedding_size, activation='relu')(Flatten()(other))
            res = tf.reduce_sum(cur * other_embed, axis=1, keepdims=True)
            res = res * cur
            res = tf.expand_dims(res, axis=1)
            result.append(res)
        result = tf.concat(result, axis=1)
        return result

In [23]:
class ImpFM(Layer):
    def __init__(self, **kwargs):
        super(ImpFM, self).__init__(**kwargs)

    def build(self, input_shape):
        self.field_size = input_shape[-2]
        self.embedding_size = input_shape[-1]
        super(ImpFM, self).build(input_shape)  # Be sure to call this somewhere!
        
    def call(self, inputs, **kwargs):
        x = inputs
        result = []
        for i in range(self.field_size):
            cur = x[:, i:i + 1]
            other = tf.concat([x[:, :i], x[:, i + 1:]], axis=1)
            other_embed = Dense(self.embedding_size, activation='relu')(Flatten()(other))
            other_embed = tf.expand_dims(other_embed, 1) 
            res = cur * other_embed
            result.append(res)
        result = tf.concat(result, axis=1)
        return result

In [21]:
# tencent senet

class SENET_V2(Layer):
    def __init__(self, **kwargs):
        super(SENET_V2, self).__init__(**kwargs)

    def build(self, input_shape):
        self.field_size = len(input_shape)
        super(SENET_V2, self).build(input_shape)  # Be sure to call this somewhere!
        
    def call(self, inputs, **kwargs):
        flat = tf.concat(inputs, axis=1)
        flat = Flatten()(flat)
        # print('field size:', self.field_size)
        # print('flat shape', flat.shape)
        # print('inputs shape', inputs[0].shape)
        weight = Dense(self.field_size, activation='sigmoid', use_bias=True)(flat)
        result = []
        for i in range(self.field_size):
            x = inputs[i] * weight[:, None, i:i + 1]
            result.append(x)
        result = tf.concat(result, axis=1)
        return result
    

# class SENET_V3(Layer):
#     def __init__(self, **kwargs):
#         super(SENET_V3, self).__init__(**kwargs)

#     def build(self, input_shape):
#         self.field_size = len(input_shape)
#         super(SENET_V3, self).build(input_shape)  # Be sure to call this somewhere!
        
#     def call(self, inputs, **kwargs):
#         # print('field size:', self.field_size)
#         result = []
#         for i in range(self.field_size):
#             x = inputs[i]
#             weight = Dense(1, activation='sigmoid', use_bias=False)(x)
#             weighted_x = x * weight
#             result.append(weighted_x)
#         result = tf.concat(result, axis=1)
#         return result

In [12]:
import itertools

class MIFM(Layer):
    def __init__(self, inter_fields=2,  **kwargs):
        self.inter_fields = inter_fields
        super(MIFM, self).__init__(**kwargs)

    def build(self, input_shape):
        self.num_fields = int(input_shape[1])
        self.embed_size = int(input_shape[2])
        super(MIFM, self).build(input_shape)  # Be sure to call this somewhere!
        
    def call(self, inputs, **kwargs):
        fea_extract = []
        for i, j in itertools.combinations(range(self.num_fields), 2):
            fea_embed_i = inputs[:, i:i + 1, :]
            # fea_embed_i = Dense(self.embed_size)(fea_embed_i)
            fea_embed_i = tf.reshape(fea_embed_i, [-1, self.inter_fields, int(self.embed_size / self.inter_fields)])
            
            fea_embed_j = inputs[:, j:j + 1, :]
            # fea_embed_j = Dense(self.embed_size)(fea_embed_j)
            fea_embed_j = tf.reshape(fea_embed_j, [-1, self.inter_fields, int(self.embed_size / self.inter_fields)])
            # [:, 2, 8]
            weight = Dense(1, activation='sigmoid')(tf.concat([fea_embed_i[:, 0], fea_embed_j[:, 0]], axis=1))
            # cross_term = tf.concat([fea_embed_i, fea_embed_j], axis=2)
            # cross_term = Dense(self.embed_size)(cross_term)
            fm = tf.reduce_sum(fea_embed_i[:, 1] * fea_embed_j[:, 1], axis=1, keep_dims=True)
            cross_term = weight * fm
        
            fea_extract.append(cross_term)
        # [:, 2 * 325, 8]
        fea_extract = tf.concat(fea_extract, axis=1)
        
        return fea_extract
    
#     def compute_output_shape(self, input_shape):
#         return (None, 1)

In [136]:
class ML(Layer):
    def __init__(self, **kwargs):
        super(ML, self).__init__(**kwargs)

    def build(self, input_shape):
        self.flatten_size = input_shape[-1]
        super(ML, self).build(input_shape)  # Be sure to call this somewhere!
        
    def call(self, inputs, **kwargs):
        dnn1 = Dense(256, activation='relu')(inputs)
        dnn1 = Dense(128, activation='relu')(dnn1)
        dnn1 = Dense(1)(dnn1)

        dnn2 = Dense(256, activation='relu')(inputs)
        dnn2 = Dense(128, activation='relu')(dnn2)
        dnn2 = Dense(1)(dnn2)

        dnn_logit = 0.5 * dnn1 + 0.5 * dnn2
        dnn = tf.sigmoid(dnn_logit)
        
        #kd_loss = 0.01 *(K.binary_crossentropy(dnn, dnn1, from_logits=True) + K.binary_crossentropy(dnn, dnn2, from_logits=True))
        #self.add_loss(kd_loss)

        return dnn

In [241]:
def dnn_model(alpha):
    input_list = Input(shape=(len(use_features),))
    embed_list = []
#     fm_list = []
#     seq_list = []
#     feature_list = []
    
    for i in range(len(sparse_features)):
        input_element = input_list[:, i:i + 1]
        embed_layer = Embedding(input_dim=input_embedding_dim_list[i], output_dim=8)(input_element)
        embed_list.append(embed_layer)
        
#     for i in range(len(sparse_features), len(use_features)):
#         input_element = input_list[:, i:i + 1]
#         embed_list.append(input_element)
        
    sparse_embed = SENET_V2()(embed_list)
        
#     for i in range(len(sparse_features)):
#         input_element = input_list[:, i:i + 1]
#         embed_layer = Embedding(input_dim=input_embedding_dim_list[i], output_dim=16)(input_element)
#         fm_list.append(embed_layer)
    
#     sparse_embed = Concatenate(name='all_embedding', axis=1)(embed_list)
    all_embed = Concatenate()([Flatten()(sparse_embed), input_list[:, len(sparse_features):]])
#     fm = FM()(sparse_embed)
#     fm = Dense(1)(fm)
    
#     lr = Dense(1)(all_embed)

#     cross_net_1 = CrossNet_V2()(all_embed)
#     cross_net_1 = Dense(1)(cross_net_1)
    
#     cross_net_2 = CrossNet_V2()(all_embed)
#     cross_net_2 = Dense(1)(cross_net_2)
    
#     cross_logit = 0.5 * cross_net_1 + 0.5 * cross_net_2
    
    dnn1 = Dense(256, activation='relu')(all_embed)
    dnn1 = Dense(128, activation='relu')(dnn1)
    dnn1 = Dense(1)(dnn1)
    
    dnn2 = Dense(256, activation='relu')(all_embed)
    dnn2 = Dense(128, activation='relu')(dnn2)
    dnn2 = Dense(1)(dnn2)
    
    dnn3 = Dense(256, activation='relu')(all_embed)
    dnn3 = Dense(128, activation='relu')(dnn3)
    dnn3 = Dense(1)(dnn3)
    
    dnn_logit = (dnn1 + dnn2 + dnn3) / 3
    dnn = tf.sigmoid(dnn_logit)
#     kd_loss = alpha * (K.binary_crossentropy(dnn, dnn1, from_logits=True) + K.binary_crossentropy(dnn, dnn2, from_logits=True))
    
#     dnn = Dense(256, activation='relu')(all_embed)
#     dnn = Dense(128, activation='relu')(dnn)
#     dnn = Dense(1)(dnn)
#     fm = Dense(1)(fm)
    
#     final = Add()([dnn_logit, cross_logit])

#     output = Dense(1, activation='sigmoid')(dnn)
    
    model = Model(inputs=[input_list], outputs=dnn)
#     model.add_loss(kd_loss)
    return model

reset_keras()
# model, kd_loss = dnn_model()
# model.summary()

In [214]:
class lossRecord(Callback):
    def __init__(self, log_dir):
        super(Callback, self).__init__()  
        self.writer = tf.summary.FileWriter(log_dir)
        self.num = 0
        
    def on_train_begin(self, logs={}):
        self.loss = []
        self.auc = []
        self.val_loss = []
        self.val_auc = []
    
    def on_batch_end(self, batch, logs={}):
        self.num + 1
        self.loss = logs.get('loss')
        self.auc = logs.get('auroc')
        self.val_loss = logs.get('val_loss')
        self.val_auc = logs.get('val_auroc')

        summary = tf.Summary()
        summary.value.add(tag='train_loss', simple_value=self.loss)
        summary.value.add(tag='train_auc', simple_value=self.auc)
        summary.value.add(tag='val_loss', simple_value=self.val_loss)
        summary.value.add(tag='val_auc', simple_value=self.val_auc)     
        self.writer.add_summary(summary, self.num)
        self.writer.flush()
        
# class Mylosscallback(Callback):
#     def __init__(self, log_dir):
#         super(Callback, self).__init__()  
#         self.writer = tf.summary.FileWriter(log_dir)
#         self.num=0
#     def on_train_begin(self, logs={}):
#         self.losses = {'batch':[], 'epoch':[]}
#         self.accuracy = {'batch':[], 'epoch':[]}
#         self.val_loss = {'batch':[], 'epoch':[]}
#         self.val_acc = {'batch':[], 'epoch':[]}
    
#     def on_batch_end(self, batch, logs={}):
#         self.num=self.num+1
#         self.losses=logs.get('loss')
#         self.accuracy=logs.get('acc')
#         self.val_loss=logs.get('val_loss')
#         self.val_acc=logs.get('val_acc')
#         print('debug success!!!')
#         summary = tf.Summary()
#         summary.value.add(tag='losses', simple_value=self.losses)
#         summary.value.add(tag='accuracy', simple_value=self.accuracy)
#         summary.value.add(tag='val_loss', simple_value=self.val_loss)
#         summary.value.add(tag='val_acc', simple_value=self.val_acc)     
#         self.writer.add_summary(summary, self.num)
#         self.writer.flush()

In [242]:
reset_keras()
batch_size = 2048 * 8
epochs = 1
model_count = 1

score_list = []

# filepath = './model/dnn/dnn_0%s.h5' % (model_count)
# checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, mode='min', save_best_only=True, save_weights_only=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, mode='min', patience=0, verbose=1)
# earlystopping = EarlyStopping(monitor='val_loss', patience=2, mode='min', min_delta=0.00002, verbose=1)
# for i in range(11):
#     log = record_logs('mln_{}'.format(i), 'mln.log')
# callbacks = [checkpoint, reduce_lr, earlystopping, log]
# losslog = lossRecord("./log")
#     callbacks = [log]
optimizer = Adam(lr=0.01)
model = dnn_model(i / 10)
# model = multi_gpu_model(model, 2)
# print(gc.collect())
# model.add_loss(tf.nn.sigmoid_cross_entropy_with_logits(labels=fold_train_y, logits=dnn))
# model.add_loss(kd_loss)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[auroc])
history = model.fit_generator(gen_data(fold_train_x, fold_train_y, batch_size=batch_size),
                              validation_data=gen_data(fold_valid_x, fold_valid_y, batch_size=batch_size),
                              epochs=epochs, verbose=1, shuffle=True, callbacks=callbacks)
# score_list.append(history)
reset_keras()
print('gc:', gc.collect())
# history = model.fit(fold_train_x, fold_train_y, batch_size=batch_size,
#                     validation_data=(fold_valid_x, fold_valid_y),
#                     epochs=epochs, verbose=1, shuffle=True, callbacks=callbacks)
# pred = model.predict(fold_valid_x, batch_size=batch_size, verbose=1)
# auc = roc_auc_score(fold_valid_y, pred)
# print(auc)

2238/2238 [==============================] - 87s 39ms/step - loss: 0.4517 - auroc: 0.7988 - val_loss: 0.4457 - val_auroc: 0.8056
gc: 0


In [225]:
fold_train_x.shape, fold_valid_x.shape

((36667392, 39), (9158656, 39))

In [246]:
((0.8055 - 0.8035) / 0.8035)  * 100

0.24891101431238355

In [222]:
fold_train_x = fold_train_x.iloc[:-5101]
fold_valid_x = fold_valid_x.iloc[:-9468]

In [ ]:
# dnn
# 2239/2239 [==============================] - 73s 32ms/step - loss: 0.4533 - auroc: 0.7971 - val_loss: 0.4471 - val_auroc: 0.8040

# deepfm
# 2239/2239 [==============================] - 76s 34ms/step - loss: 0.4527 - auroc: 0.7977 - val_loss: 0.4467 - val_auroc: 0.8044

# crossnet
# 2239/2239 [==============================] - 101s 45ms/step - loss: 0.4531 - auroc: 0.7974 - val_loss: 0.4470 - val_auroc: 0.8041

# crossnet v2
# 2239/2239 [==============================] - 270s 120ms/step - loss: 0.4507 - auroc: 0.7999 - val_loss: 0.4446 - val_auroc: 0.8069

# crossnet2 v2
# 2239/2239 [==============================] - 474s 212ms/step - loss: 0.4503 - auroc: 0.8004 - val_loss: 0.4443 - val_auroc: 0.8072

# se-crossnet2 v2
# 2239/2239 [==============================] - 483s 216ms/step - loss: 0.4501 - auroc: 0.8005 - val_loss: 0.4441 - val_auroc: 0.8074

# se-dnn 
# 2239/2239 [==============================] - 76s 34ms/step - loss: 0.4522 - auroc: 0.7983 - val_loss: 0.4465 - val_auroc: 0.8051

# se-dnn2
# 2239/2239 [==============================] - 81s 36ms/step - loss: 0.4518 - auroc: 0.7987 - val_loss: 0.4457 - val_auroc: 0.8055


In [12]:
model.fit_generator(gen_data(fold_train_x, fold_train_y, batch_size=batch_size),
                              validation_data=gen_data(fold_valid_x, fold_valid_y, batch_size=batch_size),
                              epochs=epochs, verbose=1, shuffle=True, callbacks=callbacks)

 559/2239 [======>.......................] - ETA: 1:01 - loss: 0.4481 - auroc: 0.8033
Epoch 00001: val_loss did not improve from 0.44681
2239/2239 [==============================] - 492s 220ms/step - loss: 0.4346 - auroc: 0.8177 - val_loss: 0.4481 - val_auroc: 0.8033


In [ ]:
# dnn 16embed 256dnn
# 2239/2239 [==============================] - 92s 41ms/step - loss: 0.4537 - auroc: 0.7965 - val_loss: 0.4478 - val_auroc: 0.8035

# deepfm 16embed 256dnn
# 2239/2239 [==============================] - 97s 44ms/step - loss: 0.4530 - auroc: 0.7972 - val_loss: 0.4471 - val_auroc: 0.8040

# gate dnn 16embed 256dnn
# 2239/2239 [==============================] - 96s 43ms/step - loss: 0.4529 - auroc: 0.7975 - val_loss: 0.4466 - val_auroc: 0.8046

# gate deepfm 16embed 256dnn
# 2239/2239 [==============================] - 113s 50ms/step - loss: 0.4521 - auroc: 0.7984 - val_loss: 0.4463 - val_auroc: 0.8049

# deepfm 16embed not shared 256 dnn
# 2239/2239 [==============================] - 157s 70ms/step - loss: 0.4539 - auroc: 0.7964 - val_loss: 0.4477 - val_auroc: 0.8034

# gate deepfm 8embed not shared 256 dnn
# 2239/2239 [==============================] - 161s 72ms/step - loss: 0.4520 - auroc: 0.7985 - val_loss: 0.4461 - val_auroc: 0.8051

# gate deepfm 16embed not shared 256 dnn
# 2239/2239 [==============================] - 189s 84ms/step - loss: 0.4519 - auroc: 0.7986 - val_loss: 0.4460 - val_auroc: 0.8053

# multi-head self-attention 16embed dnn
# 2239/2239 [==============================] - 126s 56ms/step - loss: 0.4565 - auroc: 0.7935 - val_loss: 0.4499 - val_auroc: 0.8009

# multi-head self-attention deepfm not shared 16embed 256 dnn
# 2239/2239 [==============================] - 224s 100ms/step - loss: 0.4576 - auroc: 0.7924 - val_loss: 0.4505 - val_auroc: 0.8010

# senet' 3layer field_size reduction_size field_size sigmoid dnn 16embed 256
# 2239/2239 [==============================] - 95s 42ms/step - loss: 0.4516 - auroc: 0.7989 - val_loss: 0.4458 - val_auroc: 0.8061

# senet' 1layer field_size sigmoid deepfm 16embed not shared 256 dnn
# 2239/2239 [==============================] - 160s 71ms/step - loss: 0.4517 - auroc: 0.7987 - val_loss: 0.4456 - val_auroc: 0.8057

# senet' 3layer field_size reduction_size field_size sigmoid deepfm 16embed not shared 256 dnn
# 2239/2239 [==============================] - 167s 74ms/step - loss: 0.4505 - auroc: 0.8002 - val_loss: 0.4444 - val_auroc: 0.8071

# ---------------------------------------------------------------------------------------------------------------------------------
# senet' dcn 
# 2239/2239 [==============================] - 207s 92ms/step - loss: 0.4518 - auroc: 0.7988 - val_loss: 0.4454 - val_auroc: 0.8059

# senet' crossfm
# 2239/2239 [==============================] - 264s 118ms/step - loss: 0.4498 - auroc: 0.8010 - val_loss: 0.4440 - val_auroc: 0.8077
# ---------------------------------------------------------------------------------------------------------------------------------
# double embedding
# 2239/2239 [==============================] - 187s 83ms/step - loss: 0.4534 - auroc: 0.7968 - val_loss: 0.4473 - val_auroc: 0.8038

In [29]:
# lr 1embed
# 2239/2239 [==============================] - 79s 35ms/step - loss: 0.4812 - auroc: 0.7619 - val_loss: 0.4775 - val_auroc: 0.7669

# True lr 1embed
# 2239/2239 [==============================] - 68s 31ms/step - loss: 0.4822 - auroc: 0.7607 - val_loss: 0.4777 - val_auroc: 0.7667

# lr 8embed
# 2239/2239 [==============================] - 81s 36ms/step - loss: 0.4811 - auroc: 0.7621 - val_loss: 0.4780 - val_auroc: 0.7664

# fm 8embed
# 2239/2239 [==============================] - 87s 39ms/step - loss: 0.4797 - auroc: 0.7652 - val_loss: 0.4731 - val_auroc: 0.7734

# fm + lr 8embed 1embed
# 2239/2239 [==============================] - 133s 59ms/step - loss: 0.4764 - auroc: 0.7683 - val_loss: 0.4702 - val_auroc: 0.7764

# deepfm 256dnn 16embed
# 2239/2239 [==============================] - 160s 72ms/step - loss: 0.4530 - auroc: 0.7974 - val_loss: 0.4471 - val_auroc: 0.8039

# dnn 256dnn 16embed
# 2239/2239 [==============================] - 143s 64ms/step - loss: 0.4538 - auroc: 0.7964 - val_loss: 0.4476 - val_auroc: 0.8035

# dnn 256 128dnn 16embed
# 2239/2239 [==============================] - 154s 69ms/step - loss: 0.4537 - auroc: 0.7966 - val_loss: 0.4474 - val_auroc: 0.8037

# dcn_v2 8embed 3layer
# 4477/4477 [==============================] - 403s 90ms/step - loss: 0.4505 - auroc: 0.8003 - val_loss: 0.4454 - val_auroc: 0.8063

# dcn_v2 8embed 200dnn 3layer
# 2239/2239 [==============================] - 364s 162ms/step - loss: 0.4506 - auroc: 0.7999 - val_loss: 0.4449 - val_auroc: 0.8066

# dcn_mixed 8emebd 200dnn 32low_rank 4num_experts 2layer_num
# 2239/2239 [==============================] - 502s 224ms/step - loss: 0.4515 - auroc: 0.7990 - val_loss: 0.4456 - val_auroc: 0.8056

# dcn_mixed 8emebd 200dnn 32low_rank 4num_experts 3layer_num
# 2239/2239 [==============================] - 680s 304ms/step - loss: 0.4517 - auroc: 0.7988 - val_loss: 0.4457 - val_auroc: 0.8055

# dcn_mixed 8emebd 200dnn 32low_rank 6num_experts 2layer_num
# 2239/2239 [==============================] - 675s 301ms/step - loss: 0.4522 - auroc: 0.7982 - val_loss: 0.4465 - val_auroc: 0.8056

(305, 1)

In [ ]:
batch_size = 2048 * 4
epochs = 20
model_count = 2

skf = StratifiedKFold(n_splits=5, random_state=seed, shuffle=True)
sub = np.zeros((test_df.shape[0], 2))
oof_pred = np.zeros((train_df.shape[0], 2))
score = []
score_list = []
count = 1

for i, (train_index, valid_index) in enumerate(skf.split(train_df, train_df['label'])):
    reset_keras()
    print('Fold: ', count)
    print("##" * 50)
    filepath = './model/dnn/dnn_0%s_0%s.h5' % (model_count, count)
    checkpoint = ModelCheckpoint(filepath, monitor='val_auroc', verbose=1, mode='max', save_best_only=True, save_weights_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_auroc', factor=0.5, mode='max', patience=2, verbose=1)
    earlystopping = EarlyStopping(monitor='val_auroc', patience=4, mode='max', restore_best_weights=True, min_delta=0.0001, verbose=1)
    callbacks = [checkpoint, reduce_lr, earlystopping]
    optimizer = Lookahead(RAdam(learning_rate=0.01), sync_period=20)
    model = dnn_model()
    if count == 1:
        model.summary()
    model = multi_gpu_model(model, 4)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[auroc])

    fold_train_x = train_df[use_features].iloc[train_index]
    fold_train_y = to_categorical(train_df['label'].iloc[train_index])
    fold_valid_x = train_df[use_features].iloc[valid_index]
    fold_valid_y = to_categorical(train_df['label'].iloc[valid_index])
    
    history = model.fit(fold_train_x, fold_train_y, validation_data=(fold_valid_x, fold_valid_y),
                        batch_size=batch_size, epochs=epochs, verbose=1, shuffle=True, callbacks=callbacks)
    model.load_weights(filepath)
    oof_pred[valid_index] = model.predict(fold_valid_x, batch_size=batch_size, verbose=1)
    pred = model.predict(test_df[use_features], batch_size=batch_size, verbose=1) / skf.n_splits
    sub += pred
    score_list.append(history.history)
    score.append(np.max(history.history['val_auroc']))
    
    del fold_train_x
    del fold_train_y
    del fold_valid_x
    del fold_valid_y
    print('gc:', gc.collect())
    
    with open('./prob/dnn/dnn_0%s_0%s_prob.h5' % (model_count, count), 'wb') as f:
        pickle.dump(pred, f)

    with open('./prob/dnn/dnn_0%s_score.h5' % model_count, 'wb') as f:
        pickle.dump(score_list, f)
        
    with open('./prob/dnn_oof_0%s_prob.h5' % model_count, 'wb') as f:
        pickle.dump(oof_pred, f)
        
    with open('./prob/dnn/dnn_0%s_prob.h5' % model_count, 'wb') as f:
        pickle.dump(sub, f)
        
    reset_keras()
    count += 1

gc.collect:  0
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:00:08.0, compute capability: 7.0
/job:localhost/replica:0/task:0/device:GPU:1 -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0000:00:09.0, compute capability: 7.0
/job:localhost/replica:0/task:0/device:GPU:2 -> device: 2, name: Tesla V100-SXM2-32GB, pci bus id: 0000:00:0a.0, compute capability: 7.0
/job:localhost/replica:0/task:0/device:GPU:3 -> device: 3, name: Tesla V100-SXM2-32GB, pci bus id: 0000:00:0b.0, compute capability: 7.0
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:1 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:2 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:3 -> device: XLA_GPU device

Fold:  1
####################################

In [39]:
model.load_weights(filepath)
pred = model.predict(test_df[use_features], batch_size=batch_size, verbose=1)

1000000/1000000 [==============================] - 6s 6us/step
